In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import torch
torch.cuda.is_available()
import torchvision.models as models
VGG16 = models.vgg16(pretrained=True)
VGG16

In [0]:
import numpy as np
from glob import glob
import torch.nn as nn
import torch.optim as optim
import torch

mushrooms_files = np.array(glob('../input/mushrooms-classification-common-genuss-images/Mushrooms/*/*'))
#dog_files = np.array(glob("/Users/hataechan/desktop/cnncifar10/muchroomclassification/mushrooms/*/*"))

import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline

from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import os
from torchvision import datasets

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

batch_size = 32

data_transforms = {'train': transforms.Compose([
                                     #transforms.RandomResizedCrop(224),
                                     #transforms.RandomHorizontalFlip(),
                                     transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                                    [0.485, 0.456, 0.406],
                                                    [0.229, 0.224, 0.225])]),
                   'valid': transforms.Compose([
                                     transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                                    [0.485, 0.456, 0.406],
                                                    [0.229, 0.224, 0.225])]),
                   'test': transforms.Compose([
                                     transforms.Resize(256),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(
                                                    [0.485, 0.456, 0.406],
                                                    [0.229, 0.224, 0.225])]),
}
data_dir = '../input/mushrooms-classification-common-genuss-images/Mushrooms/'
image_datasets = datasets.ImageFolder(data_dir, data_transforms['train'])
train_size = int(0.8 * len(image_datasets))
test_size = len(image_datasets) - train_size
valid_size = int(train_size * 0.2)
train_size = int(train_size - valid_size)
print(train_size, valid_size, test_size)

train_set, valid_set, test_set = torch.utils.data.random_split(image_datasets, [train_size, valid_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

'''
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
loaders_scratch = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True)
              for x in ['train', 'valid', 'test']}
'''
import torch
import torchvision.models as models

VGG16 = models.vgg16(pretrained=True)

use_cuda = torch.cuda.is_available()

for param in VGG16.parameters():
    param.requires_grad = False
    
if use_cuda:
    VGG16.cuda()
    
number_of_classes = len(image_datasets.classes)
classifier = nn.Sequential(nn.Linear(25088, 8192).cuda(),
                           nn.ReLU(),
                           nn.Dropout(0.5),
                           nn.Linear(8192, 1024).cuda(),
                           nn.ReLU(),
                           nn.Dropout(0.5),
                           nn.Linear(1024, number_of_classes).cuda())
VGG16.classifier = classifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(VGG16.classifier.parameters(), lr=0.001)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, train_loader, valid_loader, model, optimizer, criterion, use_cuda, save_path):

    valid_loss_min = np.Inf
    
    print(f"Batch Size: {train_loader.batch_size}\n")
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        # train the model
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if (batch_idx + 1) % 5 == 0:
                print(f'Epoch:{epoch}/{n_epochs} \tBatch:{batch_idx + 1}')
                print(f'Train Loss: {train_loss}\n')

        # validate the model
        model.eval()
        for batch_idx, (data, target) in enumerate(valid_loader):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
           
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        # save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

VGG16 = train(3, train_loader, valid_loader, VGG16, optimizer,
                       criterion, use_cuda, 'model_mush.pt')

# load the model that got the best validation accuracy
VGG16.load_state_dict(torch.load('model_mush.pt'))

def test(test_loader, model, criterion, use_cuda):
    test_loss = 0.
    correct = 0.
    total = 0.
    
    model.eval()
    for batch_idx, (data,target) in enumerate(test_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            model.requires_grad = False

        output = model(data)
        loss = criterion(output, target)
        
        test_loss += test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        output = F.softmax(output, dim=1)
        pred = output.data.max(1, keepdim=True)[1]
        
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
        print('Test Loss: {:.6f}\n'.format(test_loss))
        print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))

class_name = [item[0:] for item in image_datasets.classes]
VGG16.load_state_dict(torch.load('model_mush.pt'))

def predict_breed(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path)

    # Define transformations of image
    transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])
    # Preprocess image to 4D Tensor (.unsqueeze(0) adds a dimension)
    img_tensor = transform(img).unsqueeze(0)

    # Move tensor to GPU if available
    if use_cuda:
        img_tensor = img_tensor.cuda()
        
    ## Inference
    # Turn on evaluation mode
    VGG16.eval()
    
    # Get predicted category for image
    with torch.no_grad():
        output = VGG16(img_tensor)

    prediction = torch.argmax(output).item() 
    print(output, torch.argmax(output).item())
    # Turn off evaluation mode
    VGG16.train()
    
    # Use prediction to get dog breed
    breed = class_names[prediction]
    
    return breed

test(test_loader, VGG16, criterion, use_cuda)

def run_app(img_path):
        plt.imshow(Image.open(img_path))
        plt.show()
        print(f'This is a picture of a ... {predict_breed(img_path)}')
        print('\n-----------------------------------\n')

for file in np.hstack(mushrooms_files[:3]):
    run_app(file)

In [0]:
def predict_breed(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path)

    # Define transformations of image
    transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])
    # Preprocess image to 4D Tensor (.unsqueeze(0) adds a dimension)
    img_tensor = transform(img).unsqueeze(0)

    # Move tensor to GPU if available
    if use_cuda:
        img_tensor = img_tensor.cuda()
        
    ## Inference
    # Turn on evaluation mode
    VGG16.eval()
    
    # Get predicted category for image
    with torch.no_grad():
        output = VGG16(img_tensor)

    prediction = torch.argmax(output).item() 
    # Turn off evaluation mode
    VGG16.train()
    
    # Use prediction to get dog breed
    breed = class_names[prediction]
    
    return breed

for file in np.hstack(mushrooms_files[:3]):
    plt.show(Image.open(file))
    plt.show()
    print("breed is ",predict_breed(file))

def train(n_epochs, train_loader, valid_loader, model, optimizer, criterion, use_cuda, save_path):
    
    valid_loss_min = np.Inf
    
    print("Training...")
    
    for epoch in (1, n_epochs+1):
        
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data,target) in enumerate(train_loader):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            print("data: ",data.shape,"\n","target: ",target,"\n","output: ", output,"\n","loss: ", loss)
            loss.backward()
            optimizer.step()
            
            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)
            print("epoch: {epoch}/{n_epochs}\t batch: {batch_idx + 1}")
            print("training loss: {train_loss}")
            if batch_idx == 0:
                break
            
        model.eval()
        for batch_idx, (data, target) in enumerate(valid_loader):
            if use_cuda:
                data, target = data.cuda(), target.cuda()    
            with torch.no_grad():
                output = model(data)
                
            loss = criterion(output, target)
            
            valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss) 
            if batch_idx == 0:
                break
            
        if valid_loss < valid_loss_min:
            print("Validation loss decreased ({:.6f} --> {:.6f}). Saving model...".format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

        return model
VGG16 = train(7, train_loader, valid_loader, VGG16, optimizer, criterion, use_cuda, 'model_mush.pt')

In [124]:
VGG16.load_state_dict(torch.load('model_mush.pt'))
def test(test_loader, model, criterion, use_cuda):
    test_loss = 0.
    correct = 0.
    total = 0.
    
    model.eval()
    for batch_idx, (data,target) in enumerate(test_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            model.requires_grad = False

        output = model(data)
        loss = criterion(output, target)
        
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        output = F.softmax(output, dim=1)
        pred = output.data.max(1, keepdim=True)[1]
        
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))
        
test(test_loader, VGG16, criterion, use_cuda)

Test Loss: 0.943538


Test Accuracy: 68% (920/1343)


In [ ]:
def run_app(img_path):
        plt.imshow(Image.open(img_path))
        plt.show()
        print(f'This is a picture of a ... {predict_breed(img_path)}')
        print('\n-----------------------------------\n')
        
for file in np.hstack(mushrooms_files[48:51]):
    run_app(file)